# Interface

There are three main areas in this application: (1) a [**top toolbar**](#Toolbar) with icons and with the functionality of switch the (2) **central main frame**, and a (3) set of **lateral tabs** with specific functionalities.

<img src='images/screenshot-01.png'></img>

## Toolbar

### Development
![](images/toolbar-dev.png)  
Consist in the area with a built-in development interface, this view is complemented with the _Projects widget_ and both create a file manager and a debug interface.

### Visualizations and Stimuli delivery
![](images/toolbar-vis.png) ![](images/toolbar-sti.png)  
This button switch to an area for display data visualizations or run a stimuli delivery, in this area is possible to execute all user development extensions. 

### Documentation
![](images/toolbar-doc.png)  
Offline documentation 

### Home
![](images/toolbar-home.png)  
Home screen with quick access for common actions.

## Montage

**BCI-framework** uses the [MNE standard montages](https://mne.tools/stable/generated/mne.channels.make_standard_montage.html?highlight=montage#mne.channels.make_standard_montage) through a graphical interface to pair the channel name and the OpenBCI electrode location. All this information is automatically available through any interface and can be consulted on execution time from any extension: [Stimuli delivery](80-stimuli_delivery.ipynb), [Data analysis](70-data_analysis.ipynb), and [Data visualizations](75-data_visualizations.ipynb).

<center><img src='images/montage.gif'></img></center>

## Connection

The board can be configured through this graphical interface, almost all OpenBCI-Cyton options are available here. 

<center>
<img src='images/connection.gif'></img>
</center>

**Connection:**  
Serial device through USB dongle and WiFi through WiFi shield.

**IP:**  

  * _Port:_ Serial port address, like /dev/ttyUSB0 or COM0  
  * _IP:_ Ip address for WiFi shield, it could be in master or slave mode.


**Latency (under TCP):**  
The time in microseconds (us) between packet sends. The higher the OpenBCI sample rate, the higher the latency needed. Default is **1000us**, minimum stable is **50us**. For upper limit sample rates such as 4kHz/8kHz/16kHz, latency around **20000us** seems to really stabilize the system.

**Host:**  
On <a href="https://openbci-stream.readthedocs.io/en/latest/notebooks/A4-server-based_acquisition.html">distributed acquisition</a> this is the IP from machine where Cyton board is connected, if the board is connected in this same machine, the the IP is just localhost.

**Acquisition sample rate:**  
Over serial connections only is available 250 samples per second, over WiFi up to 16k with 8 channels, and 8k for 16 channels.

**Streaming package size:**  
The stream will attempt to write packages of this size, a lower value will reduce the latency for real-time but increase the computational processing. A high value will be easy to proceess but will have a big delay.

**Boardmode:**

  * _Default:_ Sends accelerometer data in aux bytes
  * _Debug:_ Sends serial output over the external serial port which is helpful for debugging.
  * _Analog:_ Reads from analog pins A5(D11), A6(D12) and if no wifi shield is present, then A7(D13) as well.
  * _Digital:_ Reads from analog pins D11, D12 and D17. If no wifi present then also D13 and D18.
  * _Marker:_ Turns accel off and injects markers into the stream by sending \`X where X is any char to add to the first AUX byte.

**Test signal**

  * _GND:_ Connect to internal GND (VDD - VSS)
  * _1X SLOW:_ Connect to test signal 1xAmplitude, slow pulse
  * _1X FAST:_ Connect to test signal 1xAmplitude, fast pulse
  * _DC:_ Connect to DC signal
  * _2X SLOW:_ Connect to test signal 2xAmplitude, slow pulse
  * _2X FAST:_ Connect to test signal 2xAmplitude, fast pulse

### Channel settings
**SRB1:**  
Select to connect all channel’s N inputs to SRB1. This effects all pins, and disconnects all N inputs from the ADC: SRB1_DISCONNECT (default), SRB1_CONNECT.

**SRB2:**  
Select to connect this channel’s P input to the SRB2 pin. This closes a switch between P input and SRB2 for the given channel, and allows the P input also remain connected to the ADC: SRB2_CONNECT (default), SRB2_DISCONNECT.

**Input type:**  
Select the ADC channel input source.

**Bias:**  
Select to include the channel input in BIAS generation.


### Leadoff Impedance

**PCHAN:**
Apply a current signal of 31.2 Hz into the P channels.

**NCHAN:**  
Apply a current signal of 31.2 Hz into the N channels.


## Electrode impedances

In the _Montage_ tab and when the streaming is started, the [input impedance](https://openbci-stream.readthedocs.io/en/latest/notebooks/A2-electrodes_impedance.html) can be measured in real time over the same electrode disposition. This feature must **not be activated meanwhile an acquisition**.

<center>
<img src='images/impedances.gif'></img>
</center>

<div class="alert alert-warning"> Note

  * The impedance measurement does not work correctly on the current version of Cyton Library, but there is a [pull request](https://github.com/OpenBCI/OpenBCI_Cyton_Library/pull/95) that solve this issue.
  * Is possible to use versions between V3.0.0 and V3.1.2, but you must reset the board every time before measurement and NEVER change the sample frequency.

</div>

## Records

When the streaming is active is posible to record the input signal from the _Records_ tab, even though the records can be configured [automatically from the stimuli delivery](80-stimuli_delivery.ipynb#Recording-EEG-automatically). The recorded data can be played offline and simulate an input stream or be explored with the 
[`openbci_stream.utils.HDF5Reader`](https://openbci-stream.readthedocs.io/en/latest/notebooks/07-data_storage_handler.html) module.

<center>
<img src='images/records.gif'></img>
</center>

## Annotations

Consists of simple Kafka topics that are handled in a special way by different systems. `marker` and `annotation` are defined on [OpenBCI-Stream](https://openbci-stream.readthedocs.io/en/latest/index.html), BCI-Framework introduced `command` to communicate the outputs of [Data analysis](70-data_analysis.ipynb#Send-commands,-annotations-and-feedbacks) with external actuators or systems and `feedback` to [send messagges directly to the stimuli delivery](80-stimuli_delivery.ipynb#Feedbacks). Moreover is possible to create custom topics and use it anywhere.

The BCI-Framework interface define in the _Annotations_ tab a system to read and write **markers**, **annotations** and **commands** (the feedbacks are for internal use exclusively):

<center>
<img src='images/annotations.gif'></img>
</center>

**Markers:**

The markers are used to synchronize events. Are streamed with the topic `marker`.  
Can be read from _Data analysis_ and streamed from _Stimuli delivery.

|                                  | Send             | Receive        |
|----------------------------------|------------------|----------------|
| Data analysis and Visualizations | X                | @loop_consumer |
| Stimuli delivery                 | self.send_marker | X              |


**Annotations:**

Used for describe special situations and other issues, like in `EDF` format. Are streamed with the topic `annotation`.  
Can be streamed from _Stimuli delivery_ and _Data analysis_.

|                                  | Send                 | 
|----------------------------------|----------------------|
| Data analysis and Visualizations | self.send_annotation |         
| Stimuli delivery                 | self.send_annotation |         

**Commands:**

Used to communicate with external services or actuators. Are streamed with the topic `command`.  
Can be read and stream from _Data analysis_.

|                                  | Send            | Receive              |
|----------------------------------|-------------------|---------------------|
| Data analysis and Visualizations | self.send_command | @loop_consumer      |
| Stimuli delivery                 | X                 | X                   |

**Feedbacks:**

Used to transmit messages from _Data analysis_ to _Stimuli delivery_.


|                                  | Send               | Receive             |
|----------------------------------|--------------------|---------------------|
| Data analysis and Visualizations | self.send_feedback | X                   |
| Stimuli delivery                 | X                  | self.listen_feedback|

## Properties

The properties are the way to access to all common configurations, this include OpenBCI board settings and environment constants. The following scripts can be used in any extension type, [Data analysis](70-data_analysis.ipynb), [Data visualization](75-data_visualizations.ipynb) or [Stimuli delivery](80-stimuli_delivery.ipynb).

In [ ]:
from bci_framework.extensions import properties as prop

|Property                      |Example                                                                                     |
|------------------------------|--------------------------------------------------------------------------------------------|
|prop.DPI                      | 141.37929179740024                                                                         |
|prop.CHANNELS                 | {"1": "Fp1", "2": "Fp2", "3": "T3", "4": "C3", "5": "C4", "6": "T4", "7": "O1", "8": "O2"} |
|prop.MONTAGE_NAME             | "standard_1020"                                                                            |
|prop.DAISY                    | false                                                                                      |
|prop.HOST                     | "192.168.1.1"                                                                              |
|prop.SAMPLE_RATE              | 1000                                                                                       |
|prop.STREAMING_PACKAGE_SIZE   | 100                                                                                        |
|prop.BOARDMODE                | "analog"                                                                                   |

This properties can be used, for example, to draw topoplots, name axis ticks or data slicing.